# Sample Workflow for d3rlpy Experiments

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import itertools
import math
import subprocess
import os
import d3rlpy
plt.style.use('matplotlibrc')

from Python.data_sampler import *

## Building an MDPDataset

We first read in a large batch of samples from the file. As `d3rlpy` wants it in the form (observations, actions, rewards, terminal flags), we go ahead and do that. Here's a helper function to get a dataset from a list of chunks of your choosing.

In [2]:
def get_dataset(chunks : list, batch_size=30000, 
                path="collected_data/rl_deterministic.txt") -> d3rlpy.dataset.MDPDataset :
    random.seed(0)
    samples = DataSampler(path_to_data="collected_data/rl_stochastic_coarse.txt")
    samples.setting("coarse")
    states = []
    actions = []
    rewards = []
    next_states = []
    for chunk in chunks:
        samples.use_chunk(chunk)
        samples.read_chunk()
        [statesChunk, actionsChunk, rewardsChunk, nextStatesChunk] = samples.get_batch(batch_size)
        states.append(statesChunk)
        actions.append(actionsChunk)
        rewards.append(rewardsChunk)
        next_states.append(nextStatesChunk)
    states = torch.cat(states)
    actions = torch.cat(actions)
    rewards = torch.cat(rewards)
    next_states = torch.cat(next_states)
    terminals = np.zeros(len(states))
    terminals[::1111] = 1 #episode length 100, change if necessary
    print(states.shape)
    dataset = d3rlpy.dataset.MDPDataset(states.numpy(), 
                                        actions.numpy(), 
                                        rewards.numpy(), terminals)
    return dataset

We can build the dataset from there, just like this, and split into train and test sets.

In [3]:
dataset = get_dataset([i for i in range(2000)])

start
[ 0.00000000e+00  7.95731469e+08  1.33108923e-02 -1.71999953e-02
 -2.33000134e-02  0.00000000e+00 -5.33423489e+00 -1.57091618e+00
  1.14820287e-01 -3.71075138e-01 -4.58600996e-01]
Read chunk # 1 out of 4999
start
[ 0.00000000e+00  7.95731469e+08 -2.35989108e-01 -2.71999953e-02
 -2.16000134e-02  0.00000000e+00 -5.33423489e+00 -1.57091618e+00
  5.42911681e-01  2.75094390e-01 -2.52970956e-01]
Read chunk # 2 out of 4999
start
[ 0.00000000e+00  7.95731469e+08 -2.83389108e-01 -4.83999953e-02
  6.19998658e-03  0.00000000e+00 -5.33423489e+00 -1.57091618e+00
 -2.54419292e-02  6.00000000e-01 -3.78270480e-02]
Read chunk # 3 out of 4999
start
[ 0.00000000e+00  7.95731469e+08 -7.09891077e-02  1.22000047e-02
  1.70999866e-02  0.00000000e+00 -5.33423489e+00 -1.57091618e+00
 -5.89277181e-01 -4.18905482e-01  1.70641772e-01]
Read chunk # 4 out of 4999
start
[ 0.00000000e+00  7.95731469e+08 -1.96989108e-01 -3.89999953e-02
  2.91999866e-02  0.00000000e+00 -5.33423489e+00 -1.57091618e+00
 -4.41872279

[ 0.00000000e+00  7.95731469e+08  3.91910892e-01 -2.13999953e-02
  1.43999866e-02  0.00000000e+00 -5.33423489e+00 -1.57091618e+00
  1.67501944e-01  1.05838918e-01  6.00000000e-01]
Read chunk # 70 out of 4999
start
[ 0.00000000e+00  7.95731469e+08  9.71089229e-03  3.48000047e-02
  2.89999866e-02  0.00000000e+00 -5.33423489e+00 -1.57091618e+00
 -4.01795012e-01 -2.41953066e-01  9.12517659e-02]
Read chunk # 71 out of 4999
start
[ 0.00000000e+00  7.95731469e+08 -3.61891077e-02  5.82000047e-02
  4.99998658e-03  0.00000000e+00 -5.33423489e+00 -1.57091618e+00
  4.35818187e-02 -5.77263510e-01 -6.00000000e-01]
Read chunk # 72 out of 4999
start
[ 0.00000000e+00  7.95731469e+08 -3.50889108e-01  4.14000047e-02
 -2.90001342e-03  0.00000000e+00 -5.33423489e+00 -1.57091618e+00
  4.67517925e-01  6.52870541e-02 -4.88590720e-01]
Read chunk # 73 out of 4999
start
[ 0.00000000e+00  7.95731469e+08  3.87110892e-01 -4.03999953e-02
 -2.96000134e-02  0.00000000e+00 -5.33423489e+00 -1.57091618e+00
  6.00000000e-

[ 0.00000000e+00  7.95731469e+08 -1.86789108e-01 -3.51999953e-02
 -1.60000134e-02  0.00000000e+00 -5.33423489e+00 -1.57091618e+00
 -3.72017270e-01 -3.18717408e-01  6.00000000e-01]
Read chunk # 136 out of 4999
start
[ 0.00000000e+00  7.95731469e+08 -2.78889108e-01 -1.91999953e-02
 -4.60001342e-03  0.00000000e+00 -5.33423489e+00 -1.57091618e+00
  7.18447329e-02  3.40893121e-01  1.82186680e-01]
Read chunk # 137 out of 4999
start
[ 0.00000000e+00  7.95731469e+08  3.93410892e-01 -4.01999953e-02
 -6.00013420e-04  0.00000000e+00 -5.33423489e+00 -1.57091618e+00
 -5.00932762e-01 -1.21514178e-02 -6.00000000e-01]
Read chunk # 138 out of 4999
start
[ 0.00000000e+00  7.95731469e+08 -3.99489108e-01  1.26000047e-02
  1.52999866e-02  0.00000000e+00 -5.33423489e+00 -1.57091618e+00
 -6.64036624e-02  6.00000000e-01  5.96353633e-01]
Read chunk # 139 out of 4999
start
[ 0.00000000e+00  7.95731469e+08 -3.99489108e-01 -2.57999953e-02
 -2.27000134e-02  0.00000000e+00 -5.33423489e+00 -1.57091618e+00
  3.674027

start
[ 0.00000000e+00  7.95731469e+08 -4.45389108e-01  1.16000047e-02
 -1.28000134e-02  0.00000000e+00 -5.33423489e+00 -1.57091618e+00
  6.00000000e-01  6.00000000e-01 -6.00000000e-01]
Read chunk # 209 out of 4999
start
[ 0.00000000e+00  7.95731469e+08  2.89610892e-01  3.14000047e-02
 -3.80001342e-03  0.00000000e+00 -5.33423489e+00 -1.57091618e+00
  2.86888161e-01  3.10754398e-01 -6.00000000e-01]
Read chunk # 210 out of 4999
start
[ 0.00000000e+00  7.95731469e+08 -2.86089108e-01 -1.61999953e-02
  1.34999866e-02  0.00000000e+00 -5.33423489e+00 -1.57091618e+00
 -6.68432610e-03 -6.00000000e-01  6.00000000e-01]
Read chunk # 211 out of 4999
start
[ 0.00000000e+00  7.95731469e+08  3.77510892e-01 -5.39999531e-03
  2.38999866e-02  0.00000000e+00 -5.33423489e+00 -1.57091618e+00
  4.35632829e-01  6.00000000e-01  1.65353798e-01]
Read chunk # 212 out of 4999
start
[ 0.00000000e+00  7.95731469e+08  5.86108923e-02  4.26000047e-02
  1.16999866e-02  0.00000000e+00 -5.33423489e+00 -1.57091618e+00
 -1.

[ 0.00000000e+00  7.95731469e+08  3.60110892e-01 -2.97999953e-02
 -1.43000134e-02  0.00000000e+00 -5.33423489e+00 -1.57091618e+00
 -6.00000000e-01  6.00000000e-01  6.25000554e-02]
Read chunk # 279 out of 4999
start
[ 0.00000000e+00  7.95731469e+08 -1.91289108e-01 -1.17999953e-02
 -1.31000134e-02  0.00000000e+00 -5.33423489e+00 -1.57091618e+00
 -5.24242184e-01  4.86107166e-01  2.67685047e-01]
Read chunk # 280 out of 4999
start
[ 0.00000000e+00  7.95731469e+08 -2.14689108e-01 -3.59999953e-02
  7.29998658e-03  0.00000000e+00 -5.33423489e+00 -1.57091618e+00
 -8.96737366e-02 -6.00000000e-01 -5.22606707e-01]
Read chunk # 281 out of 4999
start
[ 0.00000000e+00  7.95731469e+08  2.48210892e-01  2.82000047e-02
  1.04999866e-02  0.00000000e+00 -5.33423489e+00 -1.57091618e+00
  3.45877711e-01  6.00000000e-01  6.00000000e-01]
Read chunk # 282 out of 4999
start
[ 0.00000000e+00  7.95731469e+08 -2.20089108e-01 -1.99999953e-02
  1.88999866e-02  0.00000000e+00 -5.33423489e+00 -1.57091618e+00
  6.000000

...

[ 0.00000000e+00  7.95731469e+08  5.51089229e-03  3.00000469e-03
  7.09998658e-03  0.00000000e+00 -5.33423489e+00 -1.57091618e+00
  2.63770125e-01 -1.95049547e-01  6.00000000e-01]
Read chunk # 1738 out of 4999
start
[ 0.00000000e+00  7.95731469e+08  3.35210892e-01  6.20000469e-03
 -1.76000134e-02  0.00000000e+00 -5.33423489e+00 -1.57091618e+00
 -3.12514047e-01  4.48587912e-01  6.00000000e-01]
Read chunk # 1739 out of 4999
start
[ 0.00000000e+00  7.95731469e+08 -1.90089108e-01  3.38000047e-02
  1.15999866e-02  0.00000000e+00 -5.33423489e+00 -1.57091618e+00
 -6.00000000e-01 -6.00000000e-01  1.37654753e-02]
Read chunk # 1740 out of 4999
start
[ 0.00000000e+00  7.95731469e+08  1.66310892e-01  2.88000047e-02
 -8.80001342e-03  0.00000000e+00 -5.33423489e+00 -1.57091618e+00
  5.50212857e-01  6.00000000e-01 -6.00000000e-01]
Read chunk # 1741 out of 4999
start
[ 0.00000000e+00  7.95731469e+08 -3.94089108e-01 -2.79999953e-02
  3.39998658e-03  0.00000000e+00 -5.33423489e+00 -1.57091618e+00
  6.00

[ 0.00000000e+00  7.95731469e+08 -2.53089108e-01 -2.23999953e-02
 -2.65000134e-02  0.00000000e+00 -5.33423489e+00 -1.57091618e+00
 -5.45796232e-01  6.00000000e-01  5.22291465e-01]
Read chunk # 1808 out of 4999
start
[ 0.00000000e+00  7.95731469e+08  2.34110892e-01  2.46000047e-02
 -2.43000134e-02  0.00000000e+00 -5.33423489e+00 -1.57091618e+00
  6.00000000e-01  6.00000000e-01  6.00000000e-01]
Read chunk # 1809 out of 4999
start
[ 0.00000000e+00  7.95731469e+08  6.10108923e-02  3.64000047e-02
 -1.75000134e-02  0.00000000e+00 -5.33423489e+00 -1.57091618e+00
 -6.00000000e-01  2.81122863e-01 -6.30991631e-02]
Read chunk # 1810 out of 4999
start
[ 0.00000000e+00  7.95731469e+08 -3.94089108e-01  4.52000047e-02
  1.13999866e-02  0.00000000e+00 -5.33423489e+00 -1.57091618e+00
  6.00000000e-01 -6.49745358e-02  1.48420740e-01]
Read chunk # 1811 out of 4999
start
[ 0.00000000e+00  7.95731469e+08  1.53710892e-01 -3.13999953e-02
  2.79998658e-03  0.00000000e+00 -5.33423489e+00 -1.57091618e+00
 -6.00

Read chunk # 1881 out of 4999
start
[ 0.00000000e+00  7.95731469e+08  3.83210892e-01  1.08000047e-02
 -2.84000134e-02  0.00000000e+00 -5.33423489e+00 -1.57091618e+00
 -6.00000000e-01 -3.98740866e-01 -1.01644060e-01]
Read chunk # 1882 out of 4999
start
[ 0.00000000e+00  7.95731469e+08  1.14410892e-01 -3.43999953e-02
 -2.11000134e-02  0.00000000e+00 -5.33423489e+00 -1.57091618e+00
 -4.61322817e-01  2.57372467e-01  1.11872186e-01]
Read chunk # 1883 out of 4999
start
[ 0.00000000e+00  7.95731469e+08  2.36210892e-01 -2.21999953e-02
 -2.67000134e-02  0.00000000e+00 -5.33423489e+00 -1.57091618e+00
 -3.25049241e-02  6.00000000e-01 -6.00000000e-01]
Read chunk # 1884 out of 4999
start
[ 0.00000000e+00  7.95731469e+08  1.82810892e-01 -2.45999953e-02
  1.37999866e-02  0.00000000e+00 -5.33423489e+00 -1.57091618e+00
  3.95648565e-01 -4.47188535e-01 -4.78312842e-01]
Read chunk # 1885 out of 4999
start
[ 0.00000000e+00  7.95731469e+08  2.71108923e-02  1.14000047e-02
 -1.65000134e-02  0.00000000e+00 -5

start
[ 0.00000000e+00  7.95731469e+08  3.02210892e-01  3.80000469e-03
  2.18999866e-02  0.00000000e+00 -5.33423489e+00 -1.57091618e+00
  6.00000000e-01  8.18392211e-02  3.14575602e-01]
Read chunk # 1942 out of 4999
start
[ 0.00000000e+00  7.95731469e+08  4.61089229e-03 -1.79999953e-02
  1.61999866e-02  0.00000000e+00 -5.33423489e+00 -1.57091618e+00
  1.16385804e-01  6.00000000e-01  5.39608749e-01]
Read chunk # 1943 out of 4999
start
[ 0.00000000e+00  7.95731469e+08  4.06910892e-01 -4.77999953e-02
 -1.97000134e-02  0.00000000e+00 -5.33423489e+00 -1.57091618e+00
  5.48319135e-01 -6.00000000e-01  6.00000000e-01]
Read chunk # 1944 out of 4999
start
[ 0.00000000e+00  7.95731469e+08 -1.46889108e-01  1.04000047e-02
 -2.70001342e-03  0.00000000e+00 -5.33423489e+00 -1.57091618e+00
  6.00000000e-01 -6.00000000e-01  6.00000000e-01]
Read chunk # 1945 out of 4999
start
[ 0.00000000e+00  7.95731469e+08  5.74108923e-02 -1.21999953e-02
 -1.94000134e-02  0.00000000e+00 -5.33423489e+00 -1.57091618e+00


torch.Size([2220000, 6])


In [4]:
print("The behavior policy value statistics are:")
dataset.compute_stats()['return']

The behavior policy value statistics are:


{'mean': -266.96417,
 'std': 124.46084,
 'min': -676.4719,
 'max': 0.0,
 'histogram': (array([  1,   4,  17,  16,  37,  56,  64,  93, 136, 143, 154, 192, 219,
         184, 177, 163, 164, 150,  27,   2]),
  array([-676.4719  , -642.6483  , -608.8247  , -575.00116 , -541.17755 ,
         -507.35394 , -473.53033 , -439.70676 , -405.88315 , -372.05957 ,
         -338.23596 , -304.41235 , -270.58878 , -236.76517 , -202.94157 ,
         -169.11798 , -135.29439 , -101.47079 ,  -67.647194,  -33.823597,
            0.      ], dtype=float32))}

In [5]:
from sklearn.model_selection import train_test_split
train_episodes, test_episodes = train_test_split(dataset, test_size=0.2)

## Setting up an Algorithm

In [6]:
from d3rlpy.algos import CQL

from d3rlpy.preprocessing import MinMaxActionScaler
action_scaler = MinMaxActionScaler(minimum=-0.6, maximum=0.6)
#cql = CQL(action_scaler=action_scaler)

model = CQL(q_func_factory='mean', #qr -> quantile regression q function, but you don't have to use this
            reward_scaler='standard',
            action_scaler=action_scaler,
          actor_learning_rate=1e-5, 
          critic_learning_rate=0.0003, 
            use_gpu=False) #change it to true if you have one
model.build_with_dataset(dataset)

In [7]:
from d3rlpy.metrics.scorer import td_error_scorer
from d3rlpy.metrics.scorer import average_value_estimation_scorer
from d3rlpy.metrics.scorer import initial_state_value_estimation_scorer

# calculate metrics with test dataset
ave_error_init = average_value_estimation_scorer(model, test_episodes)
print(ave_error_init)

0.0751052818464132


In [8]:
%load_ext tensorboard
%tensorboard --logdir runs

ERROR: Failed to launch TensorBoard (exited with 1).
Contents of stderr:
Traceback (most recent call last):
  File "/home/dasc/anaconda3/envs/jbreeden3.10/bin/tensorboard", line 6, in <module>
    from tensorboard.main import run_main
  File "/home/dasc/anaconda3/envs/jbreeden3.10/lib/python3.10/site-packages/tensorboard/main.py", line 40, in <module>
    from tensorboard import default
  File "/home/dasc/anaconda3/envs/jbreeden3.10/lib/python3.10/site-packages/tensorboard/default.py", line 38, in <module>
    from tensorboard.plugins.audio import audio_plugin
  File "/home/dasc/anaconda3/envs/jbreeden3.10/lib/python3.10/site-packages/tensorboard/plugins/audio/audio_plugin.py", line 25, in <module>
    from tensorboard import plugin_util
  File "/home/dasc/anaconda3/envs/jbreeden3.10/lib/python3.10/site-packages/tensorboard/plugin_util.py", line 21, in <module>
    from tensorboard._vendor import bleach
  File "/home/dasc/anaconda3/envs/jbreeden3.10/lib/python3.10/site-packages/tensorb

In [9]:
model.fit(train_episodes,
        eval_episodes=test_episodes,
        n_epochs=40, 
        tensorboard_dir='runs',
        scorers={
            'td_error': td_error_scorer,
            'init_value': initial_state_value_estimation_scorer,
            'ave_value': average_value_estimation_scorer
        })

2022-04-21 11:50.13 [debug    ] RoundIterator is selected.
2022-04-21 11:50.13 [info     ] Directory is created at d3rlpy_logs/CQL_20220421115013
2022-04-21 11:50.13 [debug    ] Fitting action scaler...       action_scaler=min_max
2022-04-21 11:50.13 [debug    ] Fitting reward scaler...       reward_scaler=standard
2022-04-21 11:50.13 [warning  ] Skip building models since they're already built.
2022-04-21 11:50.13 [info     ] Parameters are saved to d3rlpy_logs/CQL_20220421115013/params.json params={'action_scaler': {'type': 'min_max', 'params': {'minimum': array(-0.6), 'maximum': array(0.6)}}, 'actor_encoder_factory': {'type': 'default', 'params': {'activation': 'relu', 'use_batch_norm': False, 'dropout_rate': None}}, 'actor_learning_rate': 1e-05, 'actor_optim_factory': {'optim_cls': 'Adam', 'betas': (0.9, 0.999), 'eps': 1e-08, 'weight_decay': 0, 'amsgrad': False}, 'alpha_learning_rate': 0.0001, 'alpha_optim_factory': {'optim_cls': 'Adam', 'betas': (0.9, 0.999), 'eps': 1e-08, 'weight

Epoch 1/40:   0%|          | 0/6928 [00:00<?, ?it/s]

2022-04-21 12:00.24 [info     ] CQL_20220421115013: epoch=1 step=6928 epoch=1 metrics={'time_sample_batch': 0.00033248131600042963, 'time_algorithm_update': 0.08607760863271101, 'temp_loss': 3.2389753310668, 'temp': 0.7392859111086187, 'alpha_loss': -13.069082023227462, 'alpha': 1.4248492442604852, 'critic_loss': 22.17708755668261, 'actor_loss': 40.39207670698833, 'time_step': 0.08674688960746034, 'td_error': 16.560567173508446, 'init_value': -82.84752655029297, 'ave_value': -83.19478249575641} step=6928
2022-04-21 12:00.24 [info     ] Model parameters are saved to d3rlpy_logs/CQL_20220421115013/model_6928.pt


Epoch 2/40:   0%|          | 0/6928 [00:00<?, ?it/s]

2022-04-21 12:11.49 [info     ] CQL_20220421115013: epoch=2 step=13856 epoch=2 metrics={'time_sample_batch': 0.00035565465206912705, 'time_algorithm_update': 0.09660828736445094, 'temp_loss': 0.7820279873513959, 'temp': 0.42524828426281797, 'alpha_loss': -10.031452970154472, 'alpha': 2.767355453334835, 'critic_loss': 54.5907691992458, 'actor_loss': 122.31750482554799, 'time_step': 0.09735213743208737, 'td_error': 44.08262380888173, 'init_value': -162.3415985107422, 'ave_value': -162.9516385855288} step=13856
2022-04-21 12:11.49 [info     ] Model parameters are saved to d3rlpy_logs/CQL_20220421115013/model_13856.pt


...

Epoch 39/40:   0%|          | 0/6928 [00:00<?, ?it/s]

2022-04-21 18:54.21 [info     ] CQL_20220421115013: epoch=39 step=270192 epoch=39 metrics={'time_sample_batch': 0.0003210365359282108, 'time_algorithm_update': 0.09014628548820354, 'temp_loss': 0.0022400938514923873, 'temp': 0.36381051433433975, 'alpha_loss': 0.04880675139081679, 'alpha': 2.0301015755596117, 'critic_loss': 274.2025833403128, 'actor_loss': 350.8632940199854, 'time_step': 0.09081239499340982, 'td_error': 137.92932967987878, 'init_value': -341.0992126464844, 'ave_value': -349.7714659517542} step=270192
2022-04-21 18:54.21 [info     ] Model parameters are saved to d3rlpy_logs/CQL_20220421115013/model_270192.pt


Epoch 40/40:   0%|          | 0/6928 [00:00<?, ?it/s]

2022-04-21 19:05.06 [info     ] CQL_20220421115013: epoch=40 step=277120 epoch=40 metrics={'time_sample_batch': 0.0003227231546582581, 'time_algorithm_update': 0.09107090109490357, 'temp_loss': -0.0017881930465167431, 'temp': 0.3647903068675111, 'alpha_loss': 0.063628239421584, 'alpha': 2.020732377799912, 'critic_loss': 273.50766204769013, 'actor_loss': 350.067744420252, 'time_step': 0.09173897290752886, 'td_error': 134.35094199225546, 'init_value': -337.4706115722656, 'ave_value': -346.0951132238864} step=277120
2022-04-21 19:05.06 [info     ] Model parameters are saved to d3rlpy_logs/CQL_20220421115013/model_277120.pt


[(1,
  {'time_sample_batch': 0.00033248131600042963,
   'time_algorithm_update': 0.08607760863271101,
   'temp_loss': 3.2389753310668,
   'temp': 0.7392859111086187,
   'alpha_loss': -13.069082023227462,
   'alpha': 1.4248492442604852,
   'critic_loss': 22.17708755668261,
   'actor_loss': 40.39207670698833,
   'time_step': 0.08674688960746034,
   'td_error': 16.560567173508446,
   'init_value': -82.84752655029297,
   'ave_value': -83.19478249575641}),
 (2,
  {'time_sample_batch': 0.00035565465206912705,
   'time_algorithm_update': 0.09660828736445094,
   'temp_loss': 0.7820279873513959,
   'temp': 0.42524828426281797,
   'alpha_loss': -10.031452970154472,
   'alpha': 2.767355453334835,
   'critic_loss': 54.5907691992458,
   'actor_loss': 122.31750482554799,
   'time_step': 0.09735213743208737,
   'td_error': 44.08262380888173,
   'init_value': -162.3415985107422,
   'ave_value': -162.9516385855288}),
 (3,
  {'time_sample_batch': 0.00034207487629412504,
   'time_algorithm_update': 0.090

In [10]:
model.save_model('cqlStochC2000_Ep40model_CPUOnly.pt')
model.save_policy('cqlStochC2000_Ep40_CPUOnly.pt')

/home/dasc/anaconda3/envs/jbreeden3.10/lib/python3.10/site-packages/d3rlpy/preprocessing/action_scalers.py:204: TracerWarning: torch.tensor results are registered as constants in the trace. You can safely ignore this warning if you use this function to create tensors out of constant variables that would be the same every time you call this function. In any other case, this might cause the trace to be incorrect.
  minimum = torch.tensor(
/home/dasc/anaconda3/envs/jbreeden3.10/lib/python3.10/site-packages/d3rlpy/preprocessing/action_scalers.py:207: TracerWarning: torch.tensor results are registered as constants in the trace. You can safely ignore this warning if you use this function to create tensors out of constant variables that would be the same every time you call this function. In any other case, this might cause the trace to be incorrect.
  maximum = torch.tensor(


## Off-Policy Evaluation

We do get some metrics on a test set of initial state value and average value. However, these estimates (using the critic's Q-function) of model performance are biased. They're useful for validation during training, but not much else. Instead, we fit a Q-function to the data (or a separate dataset, as I've done here) separately and evaluate the model's performance on it.

Feel free to change the chunks and number of steps.

In [11]:
# from d3rlpy.ope import FQE
# # metrics to evaluate with
# from d3rlpy.metrics.scorer import soft_opc_scorer


# ope_dataset = get_dataset([i+2000 for i in range(100)]) #change if you'd prefer different chunks
# ope_train_episodes, ope_test_episodes = train_test_split(ope_dataset, test_size=0.2)

# fqe = FQE(algo=model, action_scaler = action_scaler, use_gpu=False) #change this if you have one!
# fqe.fit(ope_train_episodes, eval_episodes=ope_test_episodes,
#         tensorboard_dir='runs',
#         n_epochs=100, n_steps_per_epoch=10000, #change if overfitting/underfitting
#         scorers={
#            'init_value': initial_state_value_estimation_scorer,
#             'ave_value': average_value_estimation_scorer,
#            'soft_opc': soft_opc_scorer(return_threshold=0)
#         })

In [12]:
# from d3rlpy.ope import FQE
# # metrics to evaluate with
# from d3rlpy.metrics.scorer import soft_opc_scorer


# ope_dataset = get_dataset([i*2 for i in range(100)], path="collected_data/rl_stochastic.txt") #change if you'd prefer different chunks
# ope_train_episodes, ope_test_episodes = train_test_split(ope_dataset, test_size=0.2)

# fqe = FQE(algo=model, action_scaler = action_scaler, use_gpu=False) #change this if you have one!
# fqe.fit(ope_train_episodes, eval_episodes=ope_test_episodes,
#         tensorboard_dir='runs',
#         n_epochs=100, n_steps_per_epoch=10000, #change if overfitting/underfitting
#         scorers={
#            'init_value': initial_state_value_estimation_scorer,
#             'ave_value': average_value_estimation_scorer,
#            'soft_opc': soft_opc_scorer(return_threshold=0)
#         })

In [13]:
# from d3rlpy.torch_utility import to_cpu
# to_cpu(model)
# model.save_policy("cqlStochpid2000Ep40CPU.pt")
# model.save_model("cqlStochpid2000Ep40modelCPU.pt")

In [14]:
# for key in dir(model):
#     module = getattr(model, key)
#     if isinstance(module, (torch.nn.Module, torch.nn.Parameter)):
#         print(yes)
#         print(key)
# dir(model)
# type(model)
# model.cpu()
# from d3rlpy.algos.torch.base import TorchImplBase
# new_model = TorchImplBase()
# from d3rlpy.torch_utility import _get_attributes
# model._device = "cpu:0"
# print(model._device)


# def my_get_state_dict(impl: Any) -> Dict[str, Any]:
#     rets = {}
#     for key in _get_attributes(impl):
#         obj = getattr(impl, key)
#         if isinstance(obj, (torch.nn.Module, torch.optim.Optimizer)):
#             if isinstance(obj, (torch.nn.Module, torch.nn.Parameter)):
#                 obj.cpu()
#             rets[key] = obj.state_dict()
#     return rets

# torch.save(my_get_state_dict(model), "my_test_model.pt")

# for key in dir(model):
#     obj = getattr(model, key)
#     if isinstance(obj, (torch.nn.Module, torch.nn.Parameter)):
#         obj.cpu()
#         print("convert to cpu")
# model.save_policy("cqlStochpid2000Ep40modelCPU.pt")

# import trace
# tracer = trace.Trace()
# tracer.run('model.save_policy("cqlStochpid2000Ep40modelCPU.pt")')